In [86]:
import numpy as np
import matplotlib.pyplot as plt
import json
import copy

# @brief Load a single json file
# @param filename The filename of the json file
# @return The json file as a dictionary
def load_single_json_file(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

# @brief Load multiple json files
# @param filenames The list of filenames to load
# @return A list of dictionaries, each representing a json file
def load_json_files(filenames):
    data = []
    for filename in filenames:
        data.append(load_single_json_file(filename))
    return data

# @brief find all filenames in a directory
# @param directory The directory to search
# @return A list of filenames in the directory
#
# This function shall search in a directory for all  
# files that end with the *.data extension. It will
# return a list of all filenames found.
def find_all_files(directory):
    import os
    files = os.listdir(directory)
    files = [f for f in files if f.endswith('.data')] 
    files = [os.path.join(directory, f) for f in files]
    return files


# @brief find and read all files
# @param directory The directory to search
# @return A list of dictionaries, each representing a json file
def get_data(directory):
    files = find_all_files(directory)
    # display the files
    data = load_json_files(files)
    # display the data
    for d in data:
        print(d)
    
    return data

#test_load_file_data = get_data('data')

# @brief Define the BER range
# @param min_pe The minimum value of the BER
# @param max_pe The maximum value of the BER
# @return The list of BER values
def define_BER_range(min_pe=1e-6, max_pe=0.5):
    ber_range = (min_pe, max_pe)
    Pe_list = np.logspace(np.log10(ber_range[0]), np.log10(ber_range[1]), num=100)
    return Pe_list

# @brief Define the Plot of the PUE vs BER
# @param min_pe The minimum value of the BER
# @param max_pe The maximum value of the BER
# @return The list of BER values and the axis of the plot
def define_plot():
    
    
    # there shall be 6 different plots 
    # left column direct calculation
    # middel column Parity Error Check - direct calculation
    # right column Parity Error Check - v2
    # first row CRC-8 
    # second row CRC-16
    # fourth row CRC-32
    fig, ax = plt.subplots(3, 3, figsize=(15, 15))
    plt.legend()
    plt.grid(True)
    # add horizontal line for worst case for each CRC type 
    R_worst_8 = 1/(2**8)
    R_worst_16 = 1/(2**16)
    R_worst_32 = 1/(2**32)
    ax[0,0].axhline(R_worst_8, color='r')  # Add horizontal line worst case
    ax[0,1].axhline(R_worst_8, color='r')  # Add horizontal line worst case
    ax[0,2].axhline(R_worst_8, color='r')  # Add horizontal line worst case
    ax[1,0].axhline(R_worst_16, color='r')  # Add horizontal line worst case
    ax[1,1].axhline(R_worst_16, color='r')  # Add horizontal line worst case
    ax[1,2].axhline(R_worst_16, color='r')  # Add horizontal line worst case
    ax[2,0].axhline(R_worst_32, color='r')  # Add horizontal line worst case
    ax[2,1].axhline(R_worst_32, color='r')  # Add horizontal line worst case
    ax[2,2].axhline(R_worst_32, color='r')  # Add horizontal line worst case

    # name each subplot
    ax[0,0].set_title('CRC-8 Direct Calculation')
    ax[0,1].set_title('CRC-8 Parity Error Check Direct Approach')
    ax[0,2].set_title('CRC-8 Parity Error Check Direct Approach V2')
    ax[1,0].set_title('CRC-16 Direct Calculation')
    ax[1,1].set_title('CRC-16 Parity Error Check Direct Approach')
    ax[1,2].set_title('CRC-16 Parity Error Check Direct Approach V2')
    ax[2,0].set_title('CRC-32 Direct Calculation')
    ax[2,1].set_title('CRC-32 Parity Error Check Direct Approach')
    ax[2,2].set_title('CRC-32 Parity Error Check Direct Approach V2')

    
    # set y-axis limits for all subplots
    #for i in range(3):
    #    ax[i,0].set_ylim(1e-21,1e-7)
        #for j in range(2):
        #    ax[i,j].set_ylim(1e-21,1e-7)
        #    #ax[i,j].set_ylabel('Probability of Undetected Error (PUE)')
        #    ax[i,j].set_xlabel('Bit Error Rate (BER)')
            


    plt.title('PUE vs BER for Different CRC Codes')
    plt.ylabel('Probability of Undetected Error (PUE)')
    plt.xlabel('Bit Error Rate (BER)')

    return ax

# @brief Create the label for the plot
# @param raw_data The raw data from the json file
# @return The label for the plot
def creat_label(raw_data):
    # label shall have to format "n:<n> <polynom> <Initial>"
    n = raw_data['nBits']
    polynom = raw_data['Polynomial']
    initial = raw_data['Initial'] 
    label = "n:{} {} {}".format(n, polynom, initial)

    return label


# @brief Add curve to a subplot
# @param Pe_list The list of BER values
# @param data The data to plot
# @param ax The axis to plot on
def add_data_toPlot(Pe_list, data, ax):
    # extract lable
    label = data['label'] 
    # extract axis via CRC type and Method
    #row = data['CRC']/8 - 1 # does not work for CRC-32 
    if data['CRC'] == 8:
        row = 0
    elif data['CRC'] == 16:
        row = 1
    else:
        row = 2
    
    if data['Method'] == 'direct':
        col = 0
    elif data['Method'] == 'direct_parityMatrix':
        col = 1
        # workaround till version 1 vs 2 is understood 
        ax[row, col+1].loglog(Pe_list, data['PUE_v2'], label=label)
        print("PUE Matrix: ", data['PUE'])
    else: 
        assert False, "Method not known"
        col = 10

    # define subplot ax 
    ax[row, col].loglog(Pe_list, data['PUE'], label=label)
    
    

## Probability of Undetected Error

Consider a binary $(n,k)$ linear code $C$ where $n$ equals the codeword lenght and $k$ the dataword length. Let $R$ denote the probability of undetected error and $P$ the transition probability.  

According to https://sci-hub.yt/10.1109/tcom.1985.1096340

$$ R(P) = \sum_{i=1}^n{A[i] \cdot P^i \cdot (1-P)^{n-i}}$$ 

$$\{A_i:0 \leq i \leq n\}$$

The weight distribution of its dual shall be denoted as $B_i$

$$ R(P) = 2^{-n-k} \cdot \sum_{i=0}^{n}{B_i\cdot(1-2\cdot P)^i-(1-P)^n}$$

$$\{B_i:0 \leq i \leq n\}$$


In [ ]:
# @brief Probability of undetected error calculation
#
# This function calculates the probability of undetected error
# @param P probability of error
# @param A array of coefficients
# @param n number of elements in A
# @return probability of undetected error
def ProbabilityOfUndetectedError(P,A,n):
    buffer = 0 
    for i in range(1,n+1):
        #single element
        buffer = A[i] * P**i * (1-P)**(n-i)
        buffer += buffer 
    
    return buffer

# @brief Probability of undetected error calculation
#
# This function calculates the probability of undetected error
# @param P probability of error
# @param B array of coefficients
# @param n number of elements in B
# @param k number of elements in B
# @return probability of undetected error
def ProbabilityOfUndetectedError_dual(P,B,n,k):
    buffer = 0 
    for i in range(1,n+1):
        #single element
        buffer = B[i] * (1-2*P)**i-(1-P)**n
        buffer += buffer 
    
    buffer = 2**(-n-k) * buffer
    return buffer
    

# since the formular is not 100% clear if the last term is part of the sum or not
# we will implement both versions
# @brief Probability of undetected error calculation
#
# This function calculates the probability of undetected error 
# @param P probability of error
# @param B array of coefficients
# @param n number of elements in B
# @param k number of elements in B
# @return probability of undetected error
def ProbabilityOfUndetectedError_dual_v2(P,B,n,k):
    buffer = 0 
    for i in range(1,n+1):
        #single element
        buffer = B[i] * (1-2*P)**i
        buffer += buffer 
    
    buffer = buffer -(1-P)**n
    buffer = 2**(-n-k) * buffer
    buffer += (1-P)**n
    return buffer

# @brief Get the PUE of a single BER
# @param data single dictonary entry
# @param P single BER
def get_single_PUE(data,BER):
    data_ = copy.deepcopy(data)
    print(data_)
    PUE = 0
    PUE_v2 = 0

    # decide the method 
    if data_['Method'] == 'direct':
        # calculate the PUE via the direct method
        PUE = ProbabilityOfUndetectedError(BER, data_['Data'], data_['nBits'])
    elif data_['Method'] == 'direct_parityMatrix':
        # calculate the PUE via Parity Error Check - Matrix
        PUE = ProbabilityOfUndetectedError_dual(BER, data_['Data'], data_['nBits'], data_['kBits'])
        PUE_v2 = ProbabilityOfUndetectedError_dual_v2(BER, data_['Data'], data_['nBits'], data_['kBits'])
    else:
        assert False, "Method not known"
        PUE = 0

    return PUE, PUE_v2


# @brief Get the PUE
# @param P The list of P values
# @param data-dictonary list
# @return data-dictonary list with PUE values
def get_PUE(data, P):
    for d in data:
        # add PUE to data dictionary
        d['PUE'] = []
        d['PUE_v2'] = [] # for the second version of the formular

        for e in P:
            PUE, PUE_v2 = get_single_PUE(d,e)
            d['PUE'].append(PUE)
            d['PUE_v2'].append(PUE_v2)
            
        
        d['label'] = creat_label(d)
    
    R_01 = get_single_PUE(data, 0.01)
    R_001 = get_single_PUE(data, 0.001)
    
    return data , [R_01,R_001]



In [92]:
# @brief Plot the PUE vs BER
# @param data The data to plot
# @param Pe_list The list of BER values
def plot_data(data, Pe_list):
    print("Weight: ", data['Data'])
    R_01 = get_single_PUE(data, 0.01)
    R_001 = get_single_PUE(data, 0.001)
    print("PUE(0.01)= ", R_01)
    print("PUE(0.001)= ", R_001)


    fig, axs = plt.subplots(1,3 , figsize=(15, 5))
    
    axs[0].set_title('CRC-{} Direct Calculation'.format(data["CRC"]))
    axs[1].set_title('CRC-{} Parity Error Matrix approach'.format(data["CRC"]))
    axs[2].set_title('CRC-{} Parity Error Matrix approach'.format(data["CRC"]))
        
    
    if data['Method'] == 'direct':
        col = 0
        print("adding ", data['label'] , " to subplot 0, ", col)
        axs[col].loglog(Pe_list, data['PUE'], label=data['label'] )
        print("PUE direct: ", data['PUE'])
    elif data['Method'] == 'direct_parityMatrix':
        col = 1
        axs[col].loglog(Pe_list, data['PUE'], label=data['label'] )
        # workaround till version 1 vs 2 is understood 
        ax[col+1].loglog(Pe_list, data['PUE_v2'], label=data['label'])
        print("PUE Matrix: ", data['PUE'])
        print("PUE_v2 Matrix: ", data['PUE_v2'])
    else: 
        assert False, "Method not known"
        col = 10

    # all subplots shall have the same x-axis label
    for ax in axs.flat:
        ax.set(xlabel='Bit Error Rate (BER)', ylabel='Probability of Undetected Error (PUE)')
        ax.grid(True)
        ax.legend()
    #plt.legend()
    #plt.grid(True)
    #plt.title('PUE vs BER')    

    plt.show()

# define the plot
#ax = define_plot()

# define the BER range
Pe_list = define_BER_range()

# get raw data 
raw_data = get_data('data') 

# calculate the Probability of Undetected Error (PUE)
data, key_points = get_PUE(raw_data, Pe_list) 

for d in data:
    #add_data_toPlot(Pe_list, d, ax)
    plot_data(d, Pe_list)

plt.show()

{'CRC': 16, 'Polynomial': '0x1021', 'Initial': '0x89ec', 'InputReflected': True, 'ResultReflected': True, 'FinalXOR': False, 'Method': 'direct', 'kBits': 16, 'rBits': 16, 'nBits': 32, 'Data': [0, 0, 0, 0, 0, 0, 16, 0, 344, 0, 2041, 0, 6704, 0, 14436, 0, 18480, 0, 14366, 0, 6768, 0, 2020, 0, 344, 0, 17, 0, 0, 0, 0, 0, 0], 'Systematic_Generator_Matrix': [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,

TypeError: list indices must be integers or slices, not str

## Explanation

## Direct approach via Error-Check-Matrix

**Parity Check Matrix $H$**

For a linear block code, the parity check matrix $H$ has dimensions $(n-k) \times n$, where $n$ is the total number of bits in each codeword, and $k$ is the number of message bits. The matrix $H$ ensures that for any valid codeword $c$, $Hc^T = 0$.

#### 1. **Binary Vector Operations**

Each row of $H$ represents a constraint that any valid codeword must satisfy. A binary vector operation in this context involves:
- **XOR Operation**: Logical XOR operation between two binary vectors (rows) is defined as:
  $$[v_1 \oplus v_2]_i = v_{1i} \oplus v_{2i} \quad \text{for all } i \text{ from } 1 \text{ to } n$$
  where $v_{1i}$ and $v_{2i}$ are the $i$-th bits of vectors $v_1$ and $v_2$ respectively, and $\oplus$ denotes the XOR operation.

#### 2. **Generating All Row Combinations**

We consider all possible subsets of rows from $H$. The number of such subsets (excluding the empty set) is $2^{n-k} - 1$. For each subset:
- **Subset Selection**: A subset of rows $R \subset \{1, 2, ..., n-k\}$ is selected. The selection can be represented as a binary number where each bit position indicates whether a row is included.
- **Vector Combination**: The corresponding rows are combined using XOR:
  $$v = \bigoplus_{i \in R} H_i$$
  where $H_i$ is the $ i $-th row of $ H $, and $ \bigoplus $ denotes the XOR operation applied across multiple vectors.

#### 3. **Calculating the Weight of Each Vector**

For each resultant vector $v$ from the combination process:
- **Weight Calculation**: The weight $w(v)$ of a vector $v$ is the number of 1's in $v$:
  $$w(v) = \sum_{i=1}^n v_i$$
  where $v_i$ is the $i$-th bit of vector $v$.

#### 4. **Building the Weight Distribution**

For every vector $v$ calculated:
- **Distribution Tallying**: Increase the count of vectors that have weight $w(v)$ in a distribution array $D$ where $D[w]$ represents the number of vectors of weight $w$:
  $$D[w(v)] = D[w(v)] + 1$$
- The array $D$ will be of size $n+1$ to account for all possible weights from 0 to $n$.

### Summary of Formulas and Steps

1. **Vector XOR**: $[v_1 \oplus v_2]_i = v_{1i} \oplus v_{2i}$.
2. **Vector Combination**: $v = \bigoplus_{i \in R} H_i$.
3. **Weight Calculation**: $w(v) = \sum_{i=1}^n v_i$.
4. **Distribution Update**: $D[w(v)] = D[w(v)] + 1$.

These steps will calculate the weight distribution for a parity check matrix, reflecting the number of vectors at each weight level that can arise from different combinations of rows, providing insights into the error detection and correction capabilities of the code.